<a href="https://colab.research.google.com/github/Jerry086/CS6140CarbonMapping/blob/main/EE_Python_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forest Carbon Mapping

## Setting Up Enviornment

Mount Google Drive to the Colab VM

In [1]:
# mount google drive if dataset is saved in it
# skip the code if dataset is uploaded or downloaded from google cloud
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Authenticate google cloud account

In [2]:
# authenticate google cloud if dataset is saved in it
# TODO: deploy T4 GPU on google cloud
from google.colab import auth
auth.authenticate_user()

Authenticate and authorize access to Earth Engine.

In [7]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=yuHr-lEYgviz8ROtDCmNDQ3QSjIy64jAsfp7fjWD0DA&tc=yo-5cHthNYvTNqujpxAx1_87ZLP7BKGd4EWpu3kRUlQ&cc=bvY7aIyACEPY8UNP00DH2TGdMFVSp2kPJalj1ovbEdo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BUA-oYsAR6VLlvFNp8e3gouALjfaAodz4mo4k_27hSQj_kDLOndPCs

Successfully saved authorization token.


## Install & import Python packages



Import Python packages used throughout this notebook.

In [3]:
# Folium setup.
import folium
print(folium.__version__)

0.14.0


In [4]:
# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)

2.12.0


## Set global variables

In [5]:
# Specify names locations for outputs in Google Drive.
# Uncomment the code for Google Drive storage.
FOLDER = 'carbon_mapping'

# Specify names locations for outputs in Google Cloud.
# BUCKET = 'cs6140'
# FOLDER = 'dataset'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# Specify feature bands to the model and the response variable.
# TODO: explore more bands
MODIS_BANDS = ['EVI']
COPERNICUS_BANDS = ['discrete_classification', 'forest_type']
TERRA_BANDS = ['Percent_Tree_Cover', 'Percent_NonTree_Vegetation']
BANDS = MODIS_BANDS + COPERNICUS_BANDS + TERRA_BANDS
RESPONSE = 'annualNPP'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches (256x256 pixels images) expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]

# Columns for input features and response
COLUMNS = [
  # Configuration for parsing a fixed-length input feature.
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
# Label each column with feature name by dictionary
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
# TODO: modify as needed
TRAIN_SIZE = 10000
EVAL_SIZE = 5000

# Specify model training parameters.
# TODO: modify as needed
BATCH_SIZE = 16
EPOCHS = 2
BUFFER_SIZE = 500
OPTIMIZER = 'Adam'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

# Visualizing Images and Image Bands

This research focuses on Forest Carbon Mapping in North America area

In [8]:
# Define the region of interest (ROI)
north_america_polygon = ee.Geometry.Polygon([
    [-168, 65],  # Northwest corner (top-left)
    [-168, 10],  # Southwest corner (bottom-left)
    [-52, 10],   # Southeast corner (bottom-right)
    [-52, 65],   # Northeast corner (top-right)
])

In [9]:
def reduce_collection(filename):
  """The filter function.
  Filter an image collection to the the year of 2018, region of North America
  Reduce temporal data by mean
  Args:
    filename: dataset filename of the collection.
  Returns:
    A representative image.
  """
  collection = ee.ImageCollection(filename)
  filtered = collection.filterDate('2018-01-01', '2018-12-31')
  reduced = filtered.mean().clip(north_america_polygon)
  return reduced

def normalize_band(image, band):
  # normalize image to [0, 1]
  image = image.select(band)
  stats = image.reduceRegion(
      reducer=ee.Reducer.minMax(),
      geometry=north_america_polygon,
      scale=250,
      bestEffort=True
  ).getInfo()
  print(stats)
  min, max = stats[band + '_min'], stats[band + '_max']
  normalized = image.unitScale(min, max)
  return normalized

NDVI and EVI bands from [MOD13Q1.061 Terra Vegetation Indices 16-Day Global 250m](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13Q1#bands).

In [ ]:
modis_image = reduce_collection("MODIS/061/MOD13Q1")
EVI = normalize_band(modis_image, 'EVI')

# Use folium to visualize the image
mapid = EVI.getMapId({'bands': ['EVI'], 'min': 0, 'max': 1, 'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ]})
# Center in Vancouver
map = folium.Map(location=[49.2827, -123.1207])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='EVI',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

{'EVI_max': 6730.608695652174, 'EVI_min': -1869}


Discrete_classification, discrete_classification-proba and forest_type bands from [Copernicus Global Land Cover Layers: CGLS-LC100 Collection 3](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#bands).

In [ ]:
copernicus_image = reduce_collection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
discrete_classification = normalize_band(copernicus_image, 'discrete_classification')
forest_type = normalize_band(copernicus_image, 'forest_type')

# Use folium to visualize the image
mapid = discrete_classification.getMapId({'bands': ['discrete_classification'], 'min': 0, 'max': 1})
map = folium.Map(location=[49.2827, -123.1207])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='discrete_classification',
  ).add_to(map)

mapid = forest_type.getMapId({'bands': ['forest_type'], 'min': 0, 'max': 1, 'palette': [
    '282828', '666000', '009900', '70663e', 'a0dc00', '929900'
  ]})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='forest_type',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

{'discrete_classification_max': 200, 'discrete_classification_min': 0}
{'forest_type_max': 5, 'forest_type_min': 0}


Percent_Tree_Cover and Percent_NonTree_Vegetation from [MOD44B.006 Terra Vegetation Continuous Fields Yearly Global 250m](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD44B#bands).

In [ ]:
terra_image = reduce_collection("MODIS/006/MOD44B")
Percent_Tree_Cover = normalize_band(terra_image, 'Percent_Tree_Cover')
Percent_NonTree_Vegetation = normalize_band(terra_image, 'Percent_NonTree_Vegetation')

# Use folium to visualize the image
mapid = Percent_Tree_Cover.getMapId({'bands': ['Percent_Tree_Cover'], 'min': 0, 'max': 1, 'palette': ['bbe029', '0a9501', '074b03']})
map = folium.Map(location=[49.2827, -123.1207])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Percent_Tree_Cover',
  ).add_to(map)

mapid = Percent_NonTree_Vegetation.getMapId({'bands': ['Percent_NonTree_Vegetation'], 'min': 0, 'max': 1, 'palette': ['bbe029', '0a9501', '074b03']})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Percent_NonTree_Vegetation',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

{'Percent_Tree_Cover_max': 84, 'Percent_Tree_Cover_min': 0}
{'Percent_NonTree_Vegetation_max': 98, 'Percent_NonTree_Vegetation_min': 0}


Response: the amount of carbon captured by plants in an ecosystem, after accounting for losses due to respiration, from [MODIS Net Primary Production CONUS](https://developers.google.com/earth-engine/datasets/catalog/UMT_NTSG_v2_MODIS_NPP#bands).

In [ ]:
# # inspect some points for validation
# points = [
#     ee.Geometry.Point(-115.16, 35.49),
#     ee.Geometry.Point(-90.54, 38.7)
# ]

# # Function to extract pixel values at specified points
# def get_pixel_values(point):
#     value_dict = npp_normalized.reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=250,bestEffort=True)
#     return value_dict

# # Loop through the points and get the pixel values
# for point in points:
#     pixel_values = get_pixel_values(point)
#     print(pixel_values.getInfo())

{'annualNPP': 0.030910581323860917}
{'annualNPP': 0.44173099255413617}


In [ ]:
npp_image = reduce_collection('UMT/NTSG/v2/MODIS/NPP')
annualNPP = normalize_band(npp_image, 'annualNPP')

mapid = annualNPP.getMapId({'bands': ['annualNPP'], 'min': 0, 'max': 1, 'palette': ['bbe029', '0a9501', '074b03']})
map = folium.Map(location=[38., -122.5])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='npp',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

{'annualNPP_max': 29278, 'annualNPP_min': 0}


Stack the 2D images (Terra Vegetation Indices and MODIS Net Primary Production) to create a single image from which samples can be taken. Convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band. To export training patches, convert a multi-band image to an array image using neighborhoodToArray(), then sample the image at points.


In [ ]:
featureStack = ee.Image.cat([
  EVI.select('EVI'),
  discrete_classification.select('discrete_classification'),
  forest_type.select('forest_type'),
  Percent_Tree_Cover.select('Percent_Tree_Cover'),
  Percent_NonTree_Vegetation.select('Percent_NonTree_Vegetation'),
  annualNPP.select('annualNPP')
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

Use some pre-made geometries to sample the stack in strategic locations. Specifically, these are hand-made polygons in which to take the 256x256 samples. Display the sampling polygons on a map, red for training polygons, blue for evaluation.

In [ ]:
# TODO: personalize polygons
usa = ee.FeatureCollection('projects/ee-jariwaladh0/assets/usa')
trainingPolys = usa.filter(ee.Filter.inList('shapeName', ['Maine', 'Alabama', 'South Carolina', 'New Hampshire','West Virginia','Vermont']));
evalPolys = usa.filter(ee.Filter.inList('shapeName', ['Georgia', 'Virginia']));
# trainingPolys = ee.FeatureCollection('projects/google/DemoTrainingGeometries')
# evalPolys = ee.FeatureCollection('projects/google/DemoEvalGeometries')

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[38., -100.], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

# Sampling

Take a sample from each polygon and merge the results into a single export.  The key step is sampling the array image at points, to get all the pixels in a 256x256 neighborhood at each point. Export a single TFRecord file that contains patches of pixel values in each record to build the training and testing data for the FCNN. Since each record potentially contains a lot of data (especially with big patches or many input bands), some manual sharding of the computation is necessary to avoid the `computed value too large` error.  Specifically, the following code takes multiple (smaller) samples within each geometry, merging the results to get a single export.

In [ ]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())

# These numbers determined experimentally.
n = 200 # Number of shards in each polygon, reduced to 200 samples
N = 2000 # Total sample size in each polygon.

# Export all the training data (in many pieces), with one task
# per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  # randomly choose one sample in each iteration
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(),
      scale = 250,
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)

  desc = TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc,
    # for Google Cloud Export
    # bucket = BUCKET,
    # fileNamePrefix = FOLDER + '/' + desc,
    # for Google Drive Export
    folder = FOLDER,
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(),
      scale = 250,
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)

  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc,
    # for Google Cloud Export
    # bucket = BUCKET,
    # fileNamePrefix = FOLDER + '/' + desc,
    # for Google Drive Export
    folder = FOLDER,
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.

In [10]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns:
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns:
    A tuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns:
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

Use the helpers to read in the training dataset.  Print the first record to check.

In [11]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns:
    A tf.data.Dataset of training data.
  """
	# directory for Google Drive
	root_dir = 'drive/My Drive/'
	glob = root_dir + FOLDER + '/' + 'training_patches' + '*'
	# glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '*'
	dataset = get_dataset(glob)
	# shuffle in n iterations, random pick one element from buffer in each iteration
	# batch in size BATCH_SIZE
	# repeat when all element are comsumed
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()
training
# print(iter(training.take(1)).next())

<_RepeatDataset element_spec=(TensorSpec(shape=(None, 256, 256, 5), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None))>

# Evaluation data

Now do the same thing to get an evaluation dataset.  Note that unlike the training dataset, the evaluation dataset has a batch size of 1, is not repeated and is not shuffled.

In [12]:
def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns:
    A tf.data.Dataset of evaluation data.
  """
	root_dir = 'drive/My Drive/'
	glob = root_dir + FOLDER + '/' + 'eval_patches' + '*'
	# glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + EVAL_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()
evaluation

<_RepeatDataset element_spec=(TensorSpec(shape=(None, 256, 256, 5), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None))>

# Model

Keras implementation of the U-Net model.  The U-Net model takes 256x256 pixel patches as input and outputs per-pixel class probability, label or a continuous output.  We can implement the model essentially unmodified, but will use mean squared error loss on the sigmoidal output since we are treating this as a regression problem, rather than a classification problem.

In [13]:
from keras.models import *
from keras.layers import *
from keras import metrics
from keras import optimizers
from keras import losses

# TODO: test other activation functions

def conv_block(input_tensor, num_filters):
	encoder = Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = BatchNormalization()(encoder)
	encoder = Activation('relu')(encoder)
	encoder = Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = BatchNormalization()(encoder)
	encoder = Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = concatenate([concat_tensor, decoder], axis=-1)
	decoder = BatchNormalization()(decoder)
	decoder = Activation('relu')(decoder)
	decoder = Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = BatchNormalization()(decoder)
	decoder = Activation('relu')(decoder)
	decoder = Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = BatchNormalization()(decoder)
	decoder = Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

# Training the model

Here we're going to train for 10 epochs.  For better accuracy, optimize this parameter, for example through [hyperparamter tuning](https://cloud.google.com/ml-engine/docs/tensorflow/using-hyperparameter-tuning).

In [14]:
m = get_model()

m.fit(
    x=training,
    epochs=EPOCHS,
    steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE),
    validation_data=evaluation,
    validation_steps=EVAL_SIZE)

# Save the weights
m.save_weights('drive/My Drive/carbon_mapping/params/normalized')

Epoch 1/2
625/625 [==============================] - 455s 652ms/step - loss: 0.0130 - root_mean_squared_error: 0.1139 - val_loss: 0.0447 - val_root_mean_squared_error: 0.2115
Epoch 2/2
625/625 [==============================] - 346s 554ms/step - loss: 0.0118 - root_mean_squared_error: 0.1086 - val_loss: 0.0515 - val_root_mean_squared_error: 0.2269


In [ ]:
# Display the model's architecture
m.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None, None,  0           []                               
                                 5)]                                                              
                                                                                                  
 conv2d_69 (Conv2D)             (None, None, None,   1472        ['input_4[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_81 (BatchN  (None, None, None,   128        ['conv2d_69[0][0]']              
 ormalization)                  32)                                                         

In [15]:
m = get_model()
# Restore the weights
m.load_weights('drive/My Drive/carbon_mapping/params/normalized')

# Evaluate the model
loss, acc = m.evaluate(x=evaluation, verbose=2, steps=1000)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))


1000/1000 - 17s - loss: 0.0392 - root_mean_squared_error: 0.1980 - 17s/epoch - 17ms/step
Restored model, accuracy: 19.80%


# Prediction

The prediction pipeline is:

1.  Export imagery on which to do predictions from Earth Engine in TFRecord format to a Cloud Storage bucket.
2.  Use the trained model to make the predictions.
3.  Write the predictions to a TFRecord file in a Cloud Storage.
4.  Upload the predictions TFRecord file to Earth Engine.


In [ ]:
def doExport(out_image_base, kernel_buffer, region):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toDrive(
    image = image.select(BANDS),
    description = out_image_base,
    folder = FOLDER,
    fileNamePrefix = out_image_base,
    region = region.getInfo()['coordinates'],
    scale = 30,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Google Drive...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [ ]:
def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'drive/My Drive/'{FOLDER}

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)

  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Google Drive.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  out_image_file = 'drive/My Drive/' + FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'impervious': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

Now there's all the code needed to run the prediction pipeline, all that remains is to specify the output region in which to do the prediction, the names of the output files, where to put them, and the shape of the outputs.  In terms of the shape, the model is trained on 256x256 patches, but can work (in theory) on any patch that's big enough with even dimensions ([reference](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf)).  Because of tile boundary artifacts, give the model slightly larger patches for prediction, then clip out the middle 256x256 patch.  This is controlled with a kernel buffer, half the size of which will extend beyond the kernel buffer.  For example, specifying a 128x128 kernel will append 64 pixels on each side of the patch, to ensure that the pixels in the output are taken from inputs completely covered by the kernel.

In [ ]:
# Output assets folder: YOUR FOLDER
user_folder = 'users/username' # INSERT YOUR FOLDER HERE.

# Base file name to use for TFRecord files and assets.
bj_image_base = 'FCNN_demo_beijing_384_'
# Half this will extend on the sides of each patch.
bj_kernel_buffer = [128, 128]
# Beijing
bj_region = ee.Geometry.Polygon(
        [[[115.9662455210937, 40.121362012835235],
          [115.9662455210937, 39.64293313749715],
          [117.01818643906245, 39.64293313749715],
          [117.01818643906245, 40.121362012835235]]], None, False)

In [ ]:
# Run the export.
doExport(bj_image_base, bj_kernel_buffer, bj_region)

In [ ]:
# Run the prediction.
doPrediction(bj_image_base, user_folder, bj_kernel_buffer, bj_region)

# Display the output

One the data has been exported, the model has made predictions and the predictions have been written to a file, and the image imported to Earth Engine, it's possible to display the resultant Earth Engine asset.  Here, display the impervious area predictions over Beijing, China.

In [ ]:
out_image = ee.Image(user_folder + '/' + bj_image_base)
mapid = out_image.getMapId({'min': 0, 'max': 1})
map = folium.Map(location=[39.898, 116.5097])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predicted impervious',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Image Math

Here we will calculate the aspect of our DEM. The aspect is the orientation of the slope, measured clockwise in degrees from 0 to 360, where 0 is north-facing, 90 is east-facing, 180 is south-facing, and 270 is west-facing.

For fun, and to demonstrate chaining methods together, we will also compute the sin of the aspect. The sin of the aspect (when using radians) represents the "eastness", with +1 being directly east and -1 being directly west.

In [ ]:
# Get the aspect (in degrees).
aspect = ee.Terrain.aspect(dem)

# Convert to radians, compute the sin of the aspect.
sinImage = aspect.divide(180).multiply(math.pi).sin()

### Display maps side-by-side 🐍

We can also display multiple maps in a cell output. Here we will display the DEM on the left and the sin of ths aspect of the DEM on the right. Here is how to (roughly) interpret the colors:

| West  | North/South | East |
| ----- | ----------- | ---- |
| green | white       | blue |

We picked Mount Shasta because of it's unique east/west facing mountain faces.

In [ ]:
map_params_mount_shasta = {
    'center': (41.40902, -122.19492),
    'zoom':10
}
map2b = Map(**map_params_mount_shasta)
map2b.addLayer(dem, {'min': 0, 'max': 4000}, 'elevation [meters]')
map2c = Map(**map_params_mount_shasta)
map2c.addLayer(
    sinImage,
    {'min': -1, 'max': 1,
     'palette':['green', 'white', 'blue']},
    'sine of aspect'
)
widgets.HBox([map2b, map2c])

The map objects have properties that can be queried (or set) from Python code. For example, the following code prints out the center coordinates (lon, lat) of the map.

In [ ]:
map2b.center

The properties of maps can be linked together, in order to sychronise the behavior. Run the following cell, then zoom and/or pan one of the maps.

In [ ]:
def syncronize_maps(map_1, map_2):
  map_center_link = widgets.link((map_1, 'center'), (map_2, 'center'))
  map_zoom_link = widgets.link((map_1, 'zoom'), (map_2, 'zoom'))

syncronize_maps(map2b, map2c)

## Image statistics

We will explore image statistics by:

1.   creating a map
2.   creating a custom geometry by adding points on the map
3.   calculating the stats of that custom geometry by calling `reduceRegion` on it.

We will ultimately answer the question, "what is the mean elevation?" in the custom geometry that we defined.



### Draw Control: Create a Custom Geometry on a Map

In this section, we will create a custom geometry on the map using `ipyleflet.DrawControl` tool. If a geometry wasn't created using the tool, a default geometry is set.

In [ ]:
# Use ipyleaflet to add the ability to draw a geometry on our map.
draw_control = ipyleaflet.DrawControl(
    rectangle={},
    polyline={},
    circlemarker={},
)
def handle_draw(target, action, geo_json):
    with output:
      output.clear_output()
      pprint(geo_json)
draw_control.on_draw(handle_draw)

map2d = Map(**map_init_params)
output = widgets.Output(layout={'border': '1px solid black', 'width':"200"})
map2d.addLayer(dem, {'min': 0, 'max': 1000}, 'elevation [meters]')
map2d.add_control(draw_control)
widgets.VBox([map2d, output])

The last geometry drawn on the map can be queries as follows:

In [ ]:
geom_clientside = draw_control.last_draw['geometry']
geom_clientside

It is possible that the preceding cell was run before any geometry was drawn on the map, so lets specify a default geometry in case that happens.

In [ ]:
# If no geometry was drawn on a map, use a default geometry.
if not geom_clientside:
  geom_clientside = {'type': 'Polygon',
    'coordinates': [[[-112.89, 36.46],
                     [-113.08, 36.18],
                     [-112.67, 36.16],
                     [-112.89, 36.46]]]}

  geom_clientside

In [ ]:
# Create an Earth Engine server-side geometry
geom = ee.Geometry(geom_clientside)

### Spatial reduction: Calculate Stats for Our Custom Geometry

We will use `reduceRegion` to calculate the mean elevation (in meters) for our custom geometry (that was created in the last section).

In [ ]:
# Compute the mean elevation in the polygon.
meanDict = dem.reduceRegion(
  reducer=ee.Reducer.mean(),
  geometry=geom,
  scale=90,
  bestEffort=True
)

# Get the mean from the dictionary and print it.
mean = meanDict.get('elevation')
print('Mean elevation', mean.getInfo())

# Image Collections

In this section we are going to use image collections to find the most cloudy and least cloudy images of Mountain View according to [Landsat 8](https://www.usgs.gov/landsat-missions/landsat-8-data-users-handbook) satelite imagery from 2016.

Specifically, we will use the [Landsat 8 Collection 2 Top of Atmosphere (TOA)](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_TOA) image collection. (TOA images have not been atmospherically corrected.)

In [ ]:
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')

In [ ]:
# Create a geometry that is specified by the Geo for Good venue in Mountain View.
point = ee.Geometry.Point(-122.0648754, 37.4225866)

# Define a default visualization parameters for the Landsat image.
landsat_rgb_viz = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.0,
    'max': 0.3,
}

# Filter by our Mountain View coordinates and by the year 2016, which was
# arbitrarily chosen.
spatialFiltered = landsat8.filterBounds(point)
temporalFiltered = spatialFiltered.filterDate('2016-01-01', '2016-12-31')

# Sort based on the amount of cloud cover.
least_cloudy_image = temporalFiltered.sort('CLOUD_COVER').first()
most_cloudy_image = temporalFiltered.sort('CLOUD_COVER', opt_ascending=False).first()

map_most_cloudy = Map(**map_init_params)
map_least_cloudy = Map()
map_most_cloudy.addLayer(most_cloudy_image, landsat_rgb_viz)
map_least_cloudy.addLayer(least_cloudy_image, landsat_rgb_viz)
syncronize_maps(map_most_cloudy, map_least_cloudy)

widgets.VBox([
    widgets.Label(f"CLOUD_COVER (most) = {most_cloudy_image.getInfo()['properties']['CLOUD_COVER']}"),
    map_most_cloudy,
    widgets.Label(f"CLOUD_COVER (least) = {least_cloudy_image.getInfo()['properties']['CLOUD_COVER']}"),
    map_least_cloudy
],
layout=widgets.Layout(max_height="600px")
)

## Compositing and Mosaicking

Compositing, masking and mosaicking are different technqiues that we use to process image collections.

**Compositing** refers to the process of aggregating individual pixel values in a collection. The median is often used in composites to remove the effects of cloud cover (bright pixels) and shadows (dark pixels).

In **mosaics**, individual images are stitched together (side by side). Often it is the images that were most recent that are stitched together. We will be using the Landsat 8 dataset. You can understand why the mosaic looks the way it does by taking a look at the [Landsat orbit](https://www.youtube.com/watch?v=yPF2jpjB3Qw).

We will first calculate the composite and the mosaic for Lansat 8 data for the year 2016 (year is arbitrarily chosen) for the point centered around the Geo for Good venue in Mountain View:

In [ ]:
# Filter by the year 2016 (arbitrarily chosen).
temporalFiltered = landsat8.filterDate('2016-01-01', '2016-12-31')
# Calculate the mosaic.
mosaic = temporalFiltered.mosaic()
# Calculate the composite by getting the median over time, for each band, in
# each pixel.
median = temporalFiltered.median()

We will display the mosaic on the right and the composite on the left:

In [ ]:
# Compare the mosaic and composite results.
map4a = Map(**map_init_params)
map4a.addLayer(mosaic, landsat_rgb_viz, 'Landsat 8 (mosaic)')
map4b = Map(**map_init_params)
map4b.addLayer(median, landsat_rgb_viz, 'Landsat 8 (median)')
syncronize_maps(map4a, map4b)
widgets.HBox([map4a, map4b])

Why does the mosaic have white lines?

In [ ]:
HTML(
  '<iframe width="640" height="385" '
  'src="https://www.youtube.com/embed/yPF2jpjB3Qw" '
  'frameborder="0"></iframe>')

## Masking

Masking pixels in an image makes those pixels transparent and excludes them from analysis. Pixels with a mask values of 0 or below will be transparent, mask values between 0 and 1 will be partially rendered, whereas mask values above 1 will be fully rendered.

In this example we will use a mask to only look at land data (exclude the water data) of the [Hansen Global Forest Change](https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2021_v1_9) dataset. This dataset is used because in the `datamask` column, water has a value of 2, land has the value 1, and 'no data' has the value 0.

In [ ]:
# Load or import the Hansen et al. forest change dataset.
hansenImage = ee.Image('UMD/hansen/global_forest_change_2021_v1_9')

# Select the land/water mask.
datamask = hansenImage.select('datamask')

# Create a binary mask. This means we are only selecting the land pixels (based
# on how the datamask column is defined in the dataset).
mask = datamask.eq(1)

# Update the composite mask with the water mask.
maskedComposite = median.updateMask(mask)

map4c = Map(**map_init_params)
map4c.addLayer(maskedComposite, landsat_rgb_viz, 'masked')
map4c

# NDVI, Mapping a Function over a Collection, Quality Mosaicking

In this section we will calculate the Normalized Difference Vegetation Index (NDVI) for Landsat 8 images.

The NDVI is used to determine how much green vegetation exists in an area. NDVI relies on green vegetation having a strong reflectance for Near Infrared (NIR) and a weak reflectance for red light.





The formula for NDVI is as follows:

\begin{align}
\text{NDVI} = \frac{\text{NIR}-\text{R}}{\text{NIR}+\text{R}}
\end{align}

where $\text{NIR}$ and $\text{R}$ are the spectral reflectance in the near-infrared and red (visible) regions, respectively (source: [wikipedia](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index)).

For Landsat 8 and 9:

\begin{align}
\text{NDVI} = \frac{\text{Band5}-\text{Band4}}{\text{Band5}+\text{Band4}}
\end{align}

where $\text{Band5}$ and $\text{Band4}$ are the corresponding Landsat bands, respectively (source: [USGS](https://www.usgs.gov/landsat-missions/landsat-normalized-difference-vegetation-index#:~:text=In%20Landsat%204%2D7%2C%20NDVI,Band%205%20%2B%20Band%204)).


Below are examples of NDVI calculated for two pieces of vegetation in different states.

![NDVI](https://earthobservatory.nasa.gov/ContentFeature/MeasuringVegetation/Images/ndvi_example.jpg)

*Image source: [earthobservatory.nasa.gov](https://earthobservatory.nasa.gov/ContentFeature/MeasuringVegetation/Images/ndvi_example.jpg)*

### Calculate NDVI on a Single Image

First we will calculate the NDVI on a single image:

In [ ]:
# Define a point of interest.
point = ee.Geometry.Point(location_lonlat)

# Get the least cloudy image in 2016.
image = ee.Image(
  landsat8.filterBounds(point)
          .filterDate('2016-01-01', '2016-12-31')
          .sort('CLOUD_COVER')
          .first()
)

NDVI can be calculated within Earth Engine as follows:

In [ ]:
nir = image.select('B5')
red = image.select('B4')
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

Let's display a map showing the `ndvi` object. We will use simple visualization  where blue is low (negative) NDVI and green is high (positive) NDVI. Water tends to result in a negative NDVI value, while clouds have NDVI values near zero.

In [ ]:
ndvi_vis_arams = {
    'bands': 'NDVI',
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

map5a = Map(**map_init_params)
map5a.addLayer(ndvi, ndvi_vis_arams, 'NDVI image')
map5a

Because calculating band ratios (such as NDVI) is commonly done as part of a remote sensing analysis workflow, Earth Engine images have a shortcut method to make this easier: `ee.Image.normalizedDifference()`.

In [ ]:
# Remove the NDVI layer
map5a.remove_layer(map5a.layers[1])

In [ ]:
# Redefine NDVI using ee.Image.normalizedDifference, and add it back again.
ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
map5a.addLayer(ndvi, ndvi_vis_arams, 'NDVI image')

## Applying NDVI to an Image Collection

In Earth Engine we can apply an algorithm that works on a single image (such as calculating NDVI) to all images in a collection. To do this, we first define a Python function that that operates on a single image. In this specfic example, the function takes an image, calculates NDVI, appends it to the image, and then returns the new image.

In [ ]:
def add_ndvi(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

To test it out, we can apply the `add_ndvi` function to a single image. *(This is a very useful pattern for testing and debugging functions that you write)*.

In [ ]:
ndvi = add_ndvi(image)

We can add this NDVI image to a map/inspector panel, and then use the inspector to confirm that the NDVI band was added.

In [ ]:
map_panel_5 = MapWithInspector(**map_init_params)
map_panel_5.map.addLayer(ndvi, ndvi_vis_arams, 'NDVI')
map_panel_5

But it is even more useful to "map" (i.e. apply) the function over all the images in an image collection:

In [ ]:
with_ndvi = landsat8.map(add_ndvi)

We can then mosaic the images together, and display it. Note that many images processed by the `add_ndvi` function are visible.

In [ ]:
map5b = Map(**map_init_params)
map5b.addLayer(with_ndvi.mosaic(), ndvi_vis_arams, 'NDVI mosaic')
map5b.zoom = 8  # zoom out a bit from the default
map5b

Note that the result is pretty noisy, because `.mosaic()` preferentialy selects the latest pixels in the collection (which often may have clouds). We will improve upon this in the next section.

## Make a greenest pixel composite

In this section we will use [`qualityMosaic`](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-qualitymosaic) to get less noisy NDVI data. `qualityMosaic` works by taking the maximum value composite for the band you provide. In our example, this means choosing the pixel with the largest NDVI value. The maximum is taken to avoid areas with clouds, which have very low NDVI.

In [ ]:
greenest = with_ndvi.qualityMosaic('NDVI')

In [ ]:
map5c = Map(**map_init_params)
map5c.addLayer(greenest, landsat_rgb_viz, 'greenest pixel mosaic')
map5c.zoom = 8  # zoom out a bit from the default
map5c

# Exporting Charts and Images





## Charting 🐍

Python has many plotting options, for example: matplotlib, bokeh, altair, plotly, etc. You can refer to [The Python Visualization Landscape](https://www.youtube.com/watch?v=FytuB8nFHPQ) talk from PyCon 2017 for more information on the plotting libraries that Python offers.

The goal of this section is to create a timeseries plot using Altair to showcase one Python plotting library.

We will create a timeseries plot of NDVI for the Geo for Good venue in Mountain View.

In [ ]:
stat_region = ee.Geometry.Point(location_lonlat)
stat_region.getInfo()

Filter the NDVI image collection to only get images that occur in the Geo for Good venue in Mountain View:

In [ ]:
filtered = with_ndvi.filterBounds(stat_region)

Create a function that takes the image and creates an `ee.Feature` with the mean and the image timestamp (to help enable timeseries plotting):

In [ ]:
def reduce_region_function(img):
  """Return a feature containing the mean value of a region and a timestamp."""

  stat = img.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=stat_region,
      scale=30
  )
  return ee.Feature(stat_region, stat).set({'millis': img.date().millis()})

The next two code blocks are helper functions that are used to get the feature properties into a dictionary and then the dictionary values into a pandas dataframe:

In [ ]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

def fc_to_dataframe(fc):
  """Converts a feature collection to a Pandas dataframe."""
  return pd.DataFrame(fc_to_dict(fc).getInfo())

:Here we apply the helper functions defined above to create a Pandas dataframe:

In [ ]:
stat_fc = (
  ee.FeatureCollection(
    filtered.map(reduce_region_function)
  ).filter(
    ee.Filter.notNull(filtered.first().bandNames())
  )
)
df = fc_to_dataframe(stat_fc)
df['timestamp'] = pd.to_datetime(df['millis'], unit='ms')
#df.head()

Next we narrow our scope of the feature collection to only focus on the bands that matter for NDVI, the `B5` (near-infrared), `B4` (red) and `NDVI` bands:

In [ ]:
keys = ['B5', 'B4', 'NDVI']
source = pd.melt(
    df[['B5', 'B4', 'NDVI', 'timestamp']],
    id_vars='timestamp',
    value_vars=keys,
    var_name='band'
)
source.head()  # display the first few values

Here we use the dataframe to plot the timeseries for B5, B4, and NDVI:

In [ ]:
alt.Chart(source).mark_line().encode(
    x='timestamp:T',
    y='value',
    color=alt.Color(
        'band',
        scale=alt.Scale(
            domain=['B4', 'B5', 'NDVI'],
            range=['red', 'purple', 'green']
        )
    ),
    tooltip=['band', 'value', 'timestamp:T']
).interactive(bind_y=False)

## Exporting Images 🐍

This section will demonstrat how to export an image, using the Python client library.

To start, create a 3-band, 8-bit, color-IR composite that we will export.

In [ ]:
visualization = greenest.visualize(**landsat_rgb_viz)

We can preview the visualization by adding it to an interative map.

In [ ]:
map_init_params_eastern_sierra = {
    'center': (37.05, -118.40),
    'zoom':8
}
map6a = Map(**map_init_params_eastern_sierra)
map6a.addLayer(visualization, {}, 'visualization test')
map6a

Next we can create a *task* that can be used to export the image. Note that a task will not run until it is *started*.

In [ ]:
asset_id = 'projects/ee-ee-test-te/assets/temp/g4g22_python_export_example'

task = ee.batch.Export.image.toAsset(
      visualization,
      description='Greenest_pixel_composite',
      assetId=asset_id,
      scale=30,
      region="""[[[-119.2, 37.9],
                  [-119.2, 36.2],
                  [-117.6, 36.2],
                  [-117.6, 37.9]]]"""  # California, East Side of Sierras
  )

The task can take ~10 minutes to finish. For this training, we already ran the task, so the `start()` call has been commented out. However, if you have updated the analysis and want to export a new image, you can uncomment the following line and run it (after first updating the `asset_id` in the previous cell).

In [ ]:
# task.start()

You can check on the status of your task by visitng the Earth Engine task manager webpage: https://code.earthengine.google.com/tasks

In [ ]:
exported_asset = ee.Image(asset_id)

map6b = Map(**map_init_params_eastern_sierra)
map6b.addLayer(exported_asset, {}, 'exported asset')
map6b